# Preprocessing Script
This notebook take preprocess the csv file generated from pre-1.ipynb. 


In [1]:
import numpy as np
import json
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import plotly.io as pio
import plotly.express as px
print(pio.renderers)
# pio.renderers.default = "notebook"

Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery']



In [2]:
input_fname = "formatted-pre-1-nisan2019.csv" 
output_fname = "formatted-pre-2-" + input_fname.split("-")[-1]
path = "./data/new/"
output_fname

'formatted-pre-2-nisan2019.csv'

In [3]:
cols = ["MachineName","SourceName","EventTime", "Message","Quality","Condition","Mask","NewState","Status"]
df = pd.read_csv(path + input_fname)
print("Column  Type")
for col in df.columns:
    print(col, type(df[col][0]))
df["EventTime"] = df["EventTime"].apply(lambda d: parse(d))
df.info()

Column  Type
MachineName <class 'str'>
ServerProgID <class 'str'>
ServerNodeName <class 'str'>
SubscriptionName <class 'str'>
SourceName <class 'str'>
EventTime <class 'str'>
EventTime_MS <class 'numpy.int64'>
Severity <class 'numpy.int64'>
Message <class 'str'>
Quality <class 'numpy.int64'>
Condition <class 'str'>
SubCondition <class 'str'>
Mask <class 'numpy.int64'>
NewState <class 'numpy.int64'>
EventType <class 'str'>
EventCategory <class 'str'>
AckReq <class 'numpy.bool_'>
ActiveTime <class 'str'>
ActiveTime_MS <class 'numpy.int64'>
Cookie <class 'numpy.int64'>
ActorID <class 'numpy.float64'>
Attributes <class 'str'>
Area <class 'str'>
Status <class 'numpy.int64'>
Shelving <class 'numpy.int64'>
Priority <class 'numpy.float64'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1277118 entries, 0 to 1277117
Data columns (total 26 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   MachineName       1277118 no

In [4]:
df.tail(10)

,MachineName,ServerProgID,ServerNodeName,SubscriptionName,SourceName,EventTime,EventTime_MS,Severity,Message,Quality,...,AckReq,ActiveTime,ActiveTime_MS,Cookie,ActorID,Attributes,Area,Status,Shelving,Priority
1277108,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,47TI931A,2019-04-08 07:34:18,1410040064,500,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,...,True,2019/04/08 07:34:18.000000000,1410040064,10591992,NaN,"[4353,""FCS0102"",""47TI931A"",""PV"",690.9027,0,"""",...",FCS0102,1,0,NaN
1277109,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,48FIX2040,2019-04-08 07:38:54,-124927232,500,48FIX2040 C206'DAN C4'E A.GAZ CPV = 170 m3/h IOP-,192,...,True,2019/04/08 07:38:54.000000000,-124927232,10591992,NaN,"[4353,""FCS0105"",""48FIX2040"",""CPV"",170.23291544...",FCS0105,1,0,NaN
1277110,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,47TI931A,2019-04-08 07:30:00,-1169959936,500,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,...,True,2019/04/08 07:30:00.000000000,-1169959936,10591992,NaN,"[4353,""FCS0102"",""47TI931A"",""PV"",690.9027,0,"""",...",FCS0102,1,0,NaN
1277111,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,48FIX2040,2019-04-08 07:33:25,880040064,500,48FIX2040 C206'DAN C4'E A.GAZ CPV = 170 m3/h IOP-,192,...,True,2019/04/08 07:33:25.000000000,880040064,10591992,NaN,"[4353,""FCS0105"",""48FIX2040"",""CPV"",170.34243118...",FCS0105,1,0,NaN
1277112,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,48TIC2027,2019-04-08 07:33:12,750040064,500,48TIC2027 E209 CIKIS REJEN DRAM PV = -6.1 C IOP-,0,...,True,2019/04/08 07:33:12.000000000,750040064,10591992,NaN,"[4353,""FCS0105"",""48TIC2027"",""PV"",-6.0650825500...",FCS0105,1,0,NaN
1277113,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,47TI931A,2019-04-08 07:31:28,-289959936,500,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,...,True,2019/04/08 07:31:28.000000000,-289959936,10591992,NaN,"[4353,""FCS0102"",""47TI931A"",""PV"",690.9423,0,"""",...",FCS0102,1,0,NaN
1277114,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,48FIX2040,2019-04-08 07:33:40,1030040064,500,48FIX2040 C206'DAN C4'E A.GAZ CPV = 170 m3/h IOP-,192,...,True,2019/04/08 07:33:40.000000000,1030040064,10591992,NaN,"[4353,""FCS0105"",""48FIX2040"",""CPV"",170.37302867...",FCS0105,1,0,NaN
1277115,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,47TI931A,2019-04-08 07:33:34,970040064,500,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,...,True,2019/04/08 07:33:34.000000000,970040064,10591992,NaN,"[4353,""FCS0102"",""47TI931A"",""PV"",690.9027,0,"""",...",FCS0102,1,0,NaN
1277116,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,47TI931A,2019-04-08 07:31:44,-129959936,500,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,...,True,2019/04/08 07:31:44.000000000,-129959936,10591992,NaN,"[4353,""FCS0102"",""47TI931A"",""PV"",690.9423,0,"""",...",FCS0102,1,0,NaN
1277117,PHD47B,Yokogawa.ExaopcAECS1.1,STN0830,Yokogawa.ExaopcAECS1.1,47TI931A,2019-04-08 07:32:09,120040064,500,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,...,True,2019/04/08 07:32:09.000000000,120040064,10591992,NaN,"[4353,""FCS0102"",""47TI931A"",""PV"",690.9423,0,"""",...",FCS0102,1,0,NaN


# Preprocessing of Alarm data

There are different types of alarm data but Excel file only contains "Process Alarm Data".

**Excel Columns Description**
* MachineName: It represent the unit (e.g.,Plant-47)
* SourceName: The names of the devices operating in a unit (e.g., controller, transmitter, and calculator block names)
    1. Five types of controllers: TIC, PIC, FIC, LIC, and AIC.
    2. Three types of transmitters: TI, FI, LI
    3. Calculator block names ?

* EventTime: The date when an alarm is appeared. It also represent when it recovered right?

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state. 

**Questions:**
1. Before alarm data analysis, if ALM, NR and RECOVER messages are seen, these rows must be deleted. Why?
2. Alarm-off 
3. shelving of alarms
4. Can alarm cannot trigger twice? 
5. 

In [5]:
sources = {}
source_conditions = {}
conditions = {} # Vel, IOP ALM, NR etc
key = ""
temp_dict = None
d = None
conditions["Recover"] = 0
for i in range(0, len(df["EventTime"])):
    key = df["SourceName"][i]
    temp_dict = {"Quality":df["Quality"][i], "Mask":df["Mask"][i], "NewState":df["NewState"][i], "Status": df["Status"][i] ,"SourceName": key,"EventTime": df["EventTime"][i], "Message":df["Message"][i],"Condition": df["Condition"][i]}
    
    if sources.get(key) != None:
        sources[key].append(temp_dict)
        
        #source conditions
        d =  source_conditions[key]
        if d.get(temp_dict["Condition"]) == None:
            d[temp_dict["Condition"]] = 0    
        d[temp_dict["Condition"]]  += 1
        d["Total"] += 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        # conditions           
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] += 1
        if temp_dict["Message"].strip().find("NR") != -1:
#             print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1
    else:
        sources[key] = []
        source_conditions[key] = {}
        sources[key].append(temp_dict)
        d = source_conditions[key]
        d[temp_dict["Condition"]] = 1
        d["Total"] = 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] +=1
        if temp_dict["Message"].strip().find("NR") != -1:
#             print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1


# sorting the source conditions dictionary based on the number of alarms related to each sensor
source_conditions = {k:v for k,v in sorted(source_conditions.items(), key=lambda item: item[1]["Total"],reverse=True)}
conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}


### Total number of disctinct SourceNames in the excel sheet.

In [6]:

print("Total Number of Sourcenames: {s}".format(s=len(sources.keys())))
print("SourceName")
for s in sources.keys():
    print(s)


Total Number of Sourcenames: 485
SourceName
47PDI1747
47FLI-042
47PIC1505
47PI123ABC
48LI013
47FIC034
47PIC2033A
47XL015-ANN
47LI003
47TXI1594
48TIC034
47TI013
47TI1506
47XL016-ANN
47FLI-037
47PI1701
48XL006-ANN
48XL002-ANN
47FIC016
47PI1710
19FI108A
47PDI003
47TI1512A
47PDI1749
48LI015
47TI931A
48FIX2040
48TIC2027
48AAHH004-ANN
48AI004
47FI1568
47TI1713
47_AMPSB_PMCC1L
47TI1723
47PIC2038
47TI260
47FIC1515
47TI1765
47TI1727
47PDI1746
47FIC1522A
47_AMPSA_PMCC1L
48FALL2003-ANN
47FIC021
19LAH101-ANN
47LIC1509
48XL012-ANN
47_AMPSC_PMCC1L
47TI1003
47TI008
47PI2002
47PI123B
47PI123A
47TI1505
SIO-18XA001
47TI1705
47TIC3520
47LIC1505
47TI003B
47TI1512C
47TI2086
48FI2021B
48XL010-ANN
47PIC1760
47XL026A-ANN
47TI134
48LI011
47TI1769
47LIC1501
47TI1512B
47TI1512ABC
47TXI1592
48XL009-ANN
47TI1540
47FI1571
47PDIC1759
47PI123C
48LI010
47PIC2064
47TI926I
47TI1722
47TI443
47FIC1524
47TI2003
47PIC1755
47TI1533
47XL1001-ANN
47TI2077
47TI1508
47TI3519
47PDI3104
47C56SUM_DT
48XL005-ANN
47G122-ANN
47FIC037


### How many times each SourceName has been triggered (activation + Deactivation)?

In [7]:
total  = 0
i =0
for key in source_conditions.keys():
    s1 = "Source: " + key + ", Total: {} | ".format(source_conditions[key]["Total"])
    s2 = ""
    total += source_conditions[key]["Total"]
    for k in source_conditions[key].keys():
        if k == "Total" or k == "Recover":
            continue
        s2 = s2 + k +" "+ str(source_conditions[key][k]) + ", "    
    s1 = s1 + s2
    print("{}. ".format(i),s1)
    i +=1
#     if i == 10:
#         break
print("\n\n Total Number of Entries in the excel:", total)
# assert(total== len(df["SourceName"]))


LO 83, HI 41, 
77.  Source: 47LIC1518, Total: 189 | LO 9, HI 158, HH 22, 
78.  Source: 47LI024, Total: 189 | HI 60, IOP 120, HH 9, 
79.  Source: 47TI416, Total: 180 | LO 24, HI 156, 
80.  Source: 47TI870D, Total: 180 | HHH 100, HTRP 80, 
81.  Source: 48XL012-ANN, Total: 179 | ALM 179, 
82.  Source: 48HA012-ANN, Total: 174 | ALM 174, 
83.  Source: 47PI1710, Total: 172 | HI 130, HH 21, LO 21, 
84.  Source: 47TXI1594, Total: 171 | HI 171, 
85.  Source: 47FIC1524, Total: 167 | IOP 167, 
86.  Source: 47FI1509, Total: 164 | LO 100, HI 64, 
87.  Source: 47FI1005, Total: 163 | IOP 115, LO 48, 
88.  Source: 47FIC1524_CALX, Total: 160 | IOP 160, 
89.  Source: 47TI3531, Total: 147 | LO 90, LL 45, HI 12, 
90.  Source: 19FI108A, Total: 135 | HI 43, VEL+ 8, LL 16, VEL- 20, LO 45, IOP 3, 
91.  Source: 47TIC2056, Total: 134 | LO 80, LL 54, 
92.  Source: 47FIC1515, Total: 129 | LO 64, HI 53, LL 3, DV- 9, 
93.  Source: 47TI1521, Total: 129 | HI 78, LO 51, 
94.  Source: 48BAL007-ANN, Total: 128 | ALM 128

### Number of times each condition appear

In [8]:
plot_dict = {} 
plot_dict["Condition"] = [k for k in conditions.keys()]
plot_dict["Freq"] = [v for v in conditions.values()]
# print(x_axis)
# print(y_axis)

for k in conditions.keys():
#     if k == "Recover":
#         continue
    print(k,conditions[k])

# print("Recover", conditions["Recover"]) ### including "NR" message

# fig = px.bar(x=plot_dict["Condition"],y=plot_dict["Freq"])

# fig.show()

Recover 633722
IOP 591778
IOP- 267647
VEL- 112634
VEL+ 100657
LL 56268
LO 38279
ALM 37765
LTRP 28006
LLL 27733
HI 6945
HHH 2901
CNF 1870
HTRP 1407
HH 1113
ANS- 1057
DV- 836
DV+ 197
MHI 11
MLO 6
ANS+ 5
OOP 3


### Displaying Messages of top SourceName alarms (oreder by EventTime)
Which message is related to which message (i.e., activation and deactivation) ?


In [9]:
# conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

i = 0
for sname in source_conditions.keys():
    
    print("=======Source {}: {}========".format(i,sname))
    single_source_alarms = sources[sname]
    single_source_alarms = [v for v in sorted(single_source_alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    
    j = 0
    for alarm in single_source_alarms:
        print("{},---,{},--- Message: {}".format(alarm["EventTime"],alarm["Condition"], alarm["Message"]))
        j += 1
        if j == 30:
            break
    i += 1
    if i == 10:
        break


6 m3/h IOP-
2019-04-01 00:05:44,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP- Recover
2019-04-01 00:05:54,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP-
2019-04-01 00:05:55,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP- Recover
2019-04-01 00:06:06,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP-
2019-04-01 00:06:08,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP- Recover
2019-04-01 00:06:55,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP-
2019-04-01 00:06:57,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP- Recover
2019-04-01 00:08:52,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP-
2019-04-01 00:08:53,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP- Recover
2019-04-01 00:09:20,---,IOP-,--- Message: 48FIX2040 C206'DAN C4'E A.GAZ CPV = 126 m3/h IOP-
2019-04-01 00:09:21,---,IOP-

### **Find the duration of activation of each Alarm i.e., duration =  DeactivationTime - ActivationTime**

Compute mean, median, mode, on the duration...

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state.


In [10]:
from datetime import timedelta
import statistics
from collections import Counter
# c = Counter([1,1,2,2,3])
# c.most_common(1)

def getDurationAndStats(alarms):
    """Returns list of dict and stats dictionary"""
#     queue = []
    conditions_queues = {}
    stats = {} 
    start_end = []
    d = None
    alarms = [v for v in sorted(alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    for alarm in alarms:
#         print(alarm["Message"])
        if conditions_queues.get(alarm["Condition"]) == None:
            conditions_queues[alarm["Condition"]] = []

        if alarm["Message"].find("Recover") == -1 and alarm["Message"].find("NR") == -1:
            conditions_queues[alarm["Condition"]].append(alarm) # enqueue the activation
        else:
            if len(conditions_queues[alarm["Condition"]])== 0:
#                 print("(", alarm["SourceName"],alarm["Message"], ")")
                continue
            d = conditions_queues[alarm["Condition"]].pop(0) # dequeue
            d = {k:v for k,v in d.items()}
            d["StartTime"] = d["EventTime"]
            d["EndTime"] = alarm["EventTime"]
            d["EndMessage"] = alarm["Message"]
            del d["EventTime"]
            
            start_end.append(d)
#     print("Remaining Elements in Queue", len(queue))
    temps = []
    for alarm in start_end:
        temps.append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
#     print(">>log ", len(start_end))
    if len(start_end) == 0:
        return None,None
    
    stats["timedeltas"] = [v for v in sorted(temps)]
    stats["mean"] =  round(statistics.mean(stats["timedeltas"]),3) 
    stats["median"] = statistics.median(stats["timedeltas"])
    stats["mode"] =  Counter(stats["timedeltas"]).most_common(1)#statistics.mode(stats["timedeltas"])
    stats["min"] = stats["timedeltas"][0]
    stats["max"] = stats["timedeltas"][-1]
    return (start_end, stats)

In [11]:
i = 0
for key in source_conditions.keys():
    d = source_conditions[key]  
    print("{}. Source: {}, # of activations= {}, # of deactivations = {}, Diff = {}".format(i,key, d["Total"]-d["Recover"], d["Recover"], (d["Total"]-d["Recover"])-d["Recover"]))
    i += 1
#     if i > 4:
#         break

print ("==================================================================================")
i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
#     if i > 10: 
#         break 
    alarms = sources[sname]
    _, stats = getDurationAndStats(alarms)
    if stats == None:
        continue
    print("{}. source:{}, Total alarms:{} | min = {}, max = {}, median = {}, mode = {}, mean = {}".format(i,sname,len(stats["timedeltas"]) ,stats["min"], stats["max"], stats["median"], stats["mode"], stats["mean"]))
    i += 1
print("Note: All the stats are in secods")

= 443.4
295. source:48XL001-ANN, Total alarms:4 | min = 18.0, max = 2598.0, median = 222.5, mode = [(18.0, 1)], mean = 765.25
296. source:47FI1540BCD, Total alarms:3 | min = 2670.0, max = 80176.0, median = 78574.0, mode = [(2670.0, 1)], mean = 53806.667
297. source:47LIC004, Total alarms:3 | min = 11725.0, max = 616442.0, median = 199319.0, mode = [(11725.0, 1)], mean = 275828.667
298. source:47TIC259, Total alarms:3 | min = 207804.0, max = 1435958.0, median = 519795.0, mode = [(207804.0, 1)], mean = 721185.667
299. source:47TI618, Total alarms:4 | min = 3291.0, max = 54260.0, median = 47825.5, mode = [(3291.0, 1)], mean = 38300.5
300. source:19LIC104, Total alarms:2 | min = 24.0, max = 24.0, median = 24.0, mode = [(24.0, 2)], mean = 24.0
301. source:47FI2010D, Total alarms:4 | min = 82.0, max = 2474.0, median = 1265.0, mode = [(82.0, 1)], mean = 1271.5
302. source:47XL026A-ANN, Total alarms:2 | min = 1034779.0, max = 2410619.0, median = 1722699.0, mode = [(1034779.0, 1)], mean = 17226

# Exporintg results to csv including communication alarms (IOPs)


In [12]:
import calendar

csv = {"SourceName":[],"StartTime":[], "EndTime": [] , "Message" : [], "RecoveryMessage":[] ,"Quality": [],"Condition":[],"Mask":[],"NewState": [],"Status": [],"TimeDelta": []} 

csv["Year"]= []
csv["Month"] = []
csv["MonthDay"] = []
csv["WeekDay"] = []
csv["Time"]= []
csv["Hour"] = []
csv["Minute"] = []
d = None
i = 1

for sname in source_conditions.keys():
#     sname = "48TIC2026"
# Break to get top 10
#     if i > 10: 
#         break
#     i += 1    

#     sname = "48TIC2026"

    alarms = sources[sname]
    alarms,stats = getDurationAndStats(alarms)
    if alarms == None:
        continue

    for alarm in alarms:
        d = alarm["StartTime"]
        csv["SourceName"].append(alarm["SourceName"]) 
        csv["StartTime"].append(alarm["StartTime"]) 
        csv["EndTime"].append(alarm["EndTime"])
        csv["TimeDelta"].append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
        csv["Message"].append(alarm["Message"])
        csv["RecoveryMessage"].append(alarm["EndMessage"])
        csv["Quality"].append(alarm["Quality"])
        csv["Condition"].append(alarm["Condition"])
        csv["Mask"].append(alarm["Mask"])
        csv["NewState"].append(alarm["NewState"])
        csv["Status"].append(alarm["Status"])
        csv["Year"].append(d.year)
        csv["Month"].append(d.month)
        csv["MonthDay"].append(d.day)
        csv["WeekDay"].append(calendar.day_name[d.weekday()])
        csv["Time"].append(d.strftime("%H:%M"))
        csv["Hour"].append(d.hour)
        csv["Minute"].append(d.minute)


In [13]:
print("Total Entries :", len(csv["Status"]))
data = pd.DataFrame(csv)
data.to_csv(path + output_fname, index = False)


Total Entries : 633568


# Exporintg results to csv withiout communication alarms (IOPs)

In [14]:

# csv = {"SourceName":[],"StartTime":[], "EndTime": [] , "Message" : [], "RecoveryMessage":[] ,"Quality": [],"Condition":[],"Mask":[],"NewState": [],"Status": [],"TimeDelta": []} 

# csv["Year"]= []
# csv["Month"] = []
# csv["MonthDay"] = []
# csv["WeekDay"] = []
# csv["Time"]= []
# csv["Hour"] = []
# csv["Minute"] = []
# d = None
# i = 1
# for sname in source_conditions.keys():

# # Break to get top 10
#     if i > 10: 
#         break
#     i += 1    

# #     sname = "48TIC2026"

#     alarms = sources[sname]
#     alarms,stats = getDurationAndStats(alarms)
#     if alarms == None:
#         continue

#     for alarm in alarms:
#         if alarm["Condition"] == "IOP" or alarm["Condition"] == "IOP-":
#             continue
#         d = alarm["StartTime"]
#         csv["SourceName"].append(alarm["SourceName"]) 
#         csv["StartTime"].append(alarm["StartTime"]) 
#         csv["EndTime"].append(alarm["EndTime"])
#         csv["TimeDelta"].append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
#         csv["Message"].append(alarm["Message"])
#         csv["RecoveryMessage"].append(alarm["EndMessage"])
#         csv["Quality"].append(alarm["Quality"])
#         csv["Condition"].append(alarm["Condition"])
#         csv["Mask"].append(alarm["Mask"])
#         csv["NewState"].append(alarm["NewState"])
#         csv["Status"].append(alarm["Status"])
#         csv["Year"].append(d.year)
#         csv["Month"].append(d.month)
#         csv["MonthDay"].append(d.day)
#         csv["WeekDay"].append(calendar.day_name[d.weekday()])
#         csv["Time"].append(d.strftime("%H:%M"))
#         csv["Hour"].append(d.hour)
#         csv["Minute"].append(d.minute)


# print("Total Entries written in to CSV :", len(csv["Status"]))
# data = pd.DataFrame(csv)
# data.to_csv("./data/stats-without-IOPs.csv", index = False)